In [7]:
# !pip3 install gdown
# import pandas as pd
# import gdown

# url = 'https://drive.google.com/uc?id=1-WZKE5xHw-3m_SL_PtOgwkzdFROIWqih'
# output = "reviews.csv"
# gdown.download(url, output, quiet=True)

In [21]:
import nltk 
import pandas as pd
nltk.download("punkt") 
from nltk.stem import WordNetLemmatizer
nltk.download('omw-1.4')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
from collections import defaultdict
from nltk.corpus import wordnet

tag_map = defaultdict(lambda : wordnet.NOUN)
tag_map['J'] = wordnet.ADJ
tag_map['V'] = wordnet.VERB
tag_map['R'] = wordnet.ADV

lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/archerheffern/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/archerheffern/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/archerheffern/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/archerheffern/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [9]:
def preprocess_with_lemmatization(document):
    lemmas = []
    tokens = nltk.word_tokenize(document.lower())
    tagged_tokens = nltk.pos_tag(tokens)
    for token, pos in tagged_tokens:
        lemmatizer_tag = tag_map[pos[0]]
        lemma = lemmatizer.lemmatize(token, pos = lemmatizer_tag)
        lemmas.append(lemma)
    return lemmas

In [30]:
good_tokens = ["like", "love", "good"]
bad_tokens = ["hate", "bad", "worst"]
df = pd.read_csv('reviews.csv')
df = df[df["Score"] > 0][:100]
def predict_Start(document, preprocess = None, good_tokens = good_tokens, bad_tokens = bad_tokens):
    sentiment = 0
    if preprocess:
        tokens = preprocess(document)
    else:
        tokens = document.split()
    for token in tokens:
        if token in good_tokens:
            sentiment += 1
        elif token in bad_tokens:
            sentiment -= 1

    if sentiment > 1:
        return 5
    elif sentiment == 1:
        return 4
    elif sentiment == 0:
        return 3
    elif sentiment == -1:
        return 2
    else:
        return 1
final_predict_stars = lambda text : predict_Start(text, preprocess=preprocess_with_lemmatization)
df['Prediction'] = df['Text'].apply(final_predict_stars)

In [28]:
df.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,Prediction
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...,5
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...,3
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...,3
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...,4
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...,3


In [31]:
correct_cases = sum(df['Prediction'] == df['Score'])
total_cases = len(df['Prediction'])
accuracy = (correct_cases/total_cases)
print(accuracy)

0.31
